In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np
import os 

In [3]:
# basics_url="https://1drv.ms/u/s!AitUWzJqKOwojwGP1xN4eYeEJfym?e=AkHUUS"
# basics = pd.read_csv(basics_url, sep='\t', low_memory=False)

basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)

akas_url="https://datasets.imdbws.com/title.akas.tsv.gz"
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)

ratings_url="https://datasets.imdbws.com/title.ratings.tsv.gz"
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)


# Preprocessing

In [4]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10011705 entries, 0 to 10011704
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 687.4+ MB


In [5]:
#replace "\N" with np.nan
basics.replace({'\\N': np.nan}, inplace=True)

In [6]:
#Eliminate movies that are null for runtimeMinutes
basics.dropna (subset = ["runtimeMinutes"], inplace = True)

In [7]:
#Eliminate movies that are null for genres
basics.dropna (subset = ["genres"], inplace = True)

In [8]:
#keep only titleType==Movie
basics=basics[(basics['titleType'] == 'movie')]

In [9]:
#keep startYear 2000-2022
##from .info, its observed its an object, so ill change dtype to int 
basics.dropna (subset = ["startYear"], inplace = True)
basics['startYear'] = basics['startYear'].astype(int)

##apply two filters to make startYear between 2000 to 2022
basics = basics[(basics['startYear'] >= 2000) & (basics['startYear'] <= 2022)]
basics['startYear'].value_counts()

2017    14404
2018    14384
2019    14136
2016    13992
2015    13501
2014    13141
2022    13020
2021    12471
2013    12407
2012    11661
2020    11629
2011    10789
2010    10219
2009     9379
2008     8171
2007     6978
2006     6543
2005     5860
2004     5219
2003     4606
2002     4144
2001     3885
2000     3652
Name: startYear, dtype: int64

In [10]:
#Eliminate movies that include "Documentary" in genre 
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]

# AKAS

In [11]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36562953 entries, 0 to 36562952
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   titleId          object
 1   ordering         int64 
 2   title            object
 3   region           object
 4   language         object
 5   types            object
 6   attributes       object
 7   isOriginalTitle  object
dtypes: int64(1), object(7)
memory usage: 2.2+ GB


In [12]:
#Replace "\N" with np.nan
akas.replace({'\\N': np.nan}, inplace=True)

In [13]:
#keep only US movies
##drop nulls
akas.dropna (subset = ['region'], inplace = True)
##apply filter
akas = akas[akas['region'] == 'US']
akas['region'].value_counts()

US    1451817
Name: region, dtype: int64

In [14]:
#Keep only US movies
##Filter the basics table down to only include the US by using the filter akas dataframe
basic_keepers =basics['tconst'].isin(akas['titleId'])

##filter basics
basics = basics[basic_keepers]

# Title Ratings

In [15]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1330841 entries, 0 to 1330840
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1330841 non-null  object 
 1   averageRating  1330841 non-null  float64
 2   numVotes       1330841 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 30.5+ MB


In [16]:
#Replace "\N" with np.nan
ratings.replace({'\\N': np.nan}, inplace=True)

In [17]:

#keep only US movies
##Filter the ratings table down to only include the US by using the filter akas dataframe
ratings_keepers =ratings['tconst'].isin(akas['titleId'])

##filter ratings
ratings = ratings[ratings_keepers]

# Overview

In [18]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86960 entries, 34802 to 10011471
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tconst          86960 non-null  object
 1   titleType       86960 non-null  object
 2   primaryTitle    86960 non-null  object
 3   originalTitle   86960 non-null  object
 4   isAdult         86960 non-null  object
 5   startYear       86960 non-null  int32 
 6   endYear         0 non-null      object
 7   runtimeMinutes  86960 non-null  object
 8   genres          86960 non-null  object
dtypes: int32(1), object(8)
memory usage: 6.3+ MB


In [19]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1451817 entries, 5 to 36562697
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1451817 non-null  object
 1   ordering         1451817 non-null  int64 
 2   title            1451817 non-null  object
 3   region           1451817 non-null  object
 4   language         4012 non-null     object
 5   types            981374 non-null   object
 6   attributes       46995 non-null    object
 7   isOriginalTitle  1450475 non-null  object
dtypes: int64(1), object(7)
memory usage: 99.7+ MB


In [20]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 503723 entries, 0 to 1330816
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         503723 non-null  object 
 1   averageRating  503723 non-null  float64
 2   numVotes       503723 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 15.4+ MB


# Save

In [21]:
#make data folder 
os.makedirs('Data/', exist_ok = True)

# check
os.listdir('Data/')

['.ipynb_checkpoints']

In [22]:
## Save current dataframe to file.
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

# Open saved file and preview again
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama


In [23]:
## Save current dataframe to file.
akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)

# Open saved file and preview again
akas = pd.read_csv("Data/title_akas.csv.gz", low_memory = False)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


In [24]:
## Save current dataframe to file.
ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)

# Open saved file and preview again
ratings = pd.read_csv("Data/title_ratings.csv.gz", low_memory = False)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1987
1,tt0000002,5.8,265
2,tt0000005,6.2,2631
3,tt0000006,5.1,182
4,tt0000007,5.4,824
